# Анализ текстов. Второе задание "Кластеризация писем"

In [1]:
import pandas as pd
import numpy as np

In [2]:
ds = pd.read_csv("Emails.csv")

In [4]:
messages = ds["ExtractedBodyText"]
print len(messages)

7945


Предобработка.
Во-первых, есть много не нужных сообщений. Типа ok! Yep! Pls

In [5]:
texts = [text.decode('utf-8') for text in messages.astype(str).tolist() if (len(text) > 10)]

In [6]:
print texts[:5]
print len(texts)

[u'B6\nThursday, March 3, 2011 9:45 PM\nH: Latest How Syria is aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nMarch 3, 2011\nFor: Hillary', u'H <hrod17@clintonemail.com>\nFriday, March 11, 2011 1:36 PM\nHuma Abedin\nFw: H: Latest: How Syria is aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx\nPis print.', u"Pis print.\n-\u2022-...-^\nH < hrod17@clintonernailcom>\nWednesday, September 12, 2012 2:11 PM\n\xb0Russorv@state.gov'\nFw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots\nFrom [meat)\nSent: Wednesday, September 12, 2012 01:00 PM\nTo: 11\nSubject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent from my Verizon Wireless 4G LTE DROID\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/1

Во-вторых, попробуем убрать стоп слова

In [7]:
# Убираем стоп-слова
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
texts_without_stopwords = []
for sample in texts:
    texts_without_stopwords.append(' '.join([word for word in sample.split(' ') if word.lower() not in stop_words]))

In [8]:
print texts_without_stopwords[:5]

[u'B6\nThursday, March 3, 2011 9:45 PM\nH: Latest Syria aiding Qaddafi more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nMarch 3, 2011\nFor: Hillary', u'H <hrod17@clintonemail.com>\nFriday, March 11, 2011 1:36 PM\nHuma Abedin\nFw: H: Latest: Syria aiding Qaddafi more... Sid\nhrc memo syria aiding libya 030311.docx\nPis print.', u"Pis print.\n-\u2022-...-^\nH < hrod17@clintonernailcom>\nWednesday, September 12, 2012 2:11 PM\n\xb0Russorv@state.gov'\nFw: Meet Right-Wing Extremist Behind Anti-fvluslim Film Sparked Deadly Riots\nFrom [meat)\nSent: Wednesday, September 12, 2012 01:00 PM\nTo: 11\nSubject: Meet Right Wing Extremist Behind Anti-Muslim Film Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent Verizon Wireless 4G LTE DROID\nU.S. Department State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED HOUSE SELECT BENGHAZ

Top bigramms

In [9]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk import FreqDist
tokens=nltk.word_tokenize(''.join(texts_without_stopwords))
bigrams=ngrams(tokens,2)
freq_bigrams = nltk.FreqDist(bigrams)

In [10]:
for bigram, value in freq_bigrams.most_common(30):
    print bigram, value

(u'.', u"''") 1387
(u'.', u'``') 1245
(u',', u"''") 1062
(u'.', u'The') 716
(u'Secretary', u"'s") 676
(u',', u'2010') 596
(u',', u'2009') 573
(u'@', u'state.gov') 534
(u"'s", u'Office') 483
(u'.', u'I') 467
(u'No', u'.') 461
(u'United', u'States') 445
(u'State', u'Department') 440
(u'White', u'House') 411
(u'state.gov', u'>') 399
(u',', u'``') 398
(u'I', u"'m") 374
(u'do', u"n't") 359
(u'Re', u':') 325
(u')', u'.') 296
(u'Department', u'State') 287
(u'it', u"'s") 285
(u')', u',') 282
(u'it', u'.') 276
(u'pm', u'Secretary') 273
(u'Obama', u"'s") 266
(u'.', u'-') 265
(u'.', u'In') 265
(u'U.S.', u'Department') 261
(u',', u'the') 246


Коллокации из 2х слов

In [11]:
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(5)

In [12]:
print finder.nbest(bigram_measures.pmi, 20)

[(u'BINYAMIN', u'NETANYAHU'), (u'Chico', u'Menashe'), (u'Deep', u'Throat'), (u'HAMAD', u'BIN'), (u'Im', u'Tirtzu'), (u'Buenos', u'Aires'), (u'Belief', u'Blog'), (u'Elliott', u'Trudeau'), (u'Montreal-Pierre', u'Elliott'), (u'USNS', u'Comfort'), (u'ESMTP', u'id'), (u'Generating', u'server'), (u'Sensitive', u'Unclassified'), (u'Judea', u'Samaria'), (u'Iwo', u'Jima'), (u'Vital', u'Voices'), (u'Cape', u'Verde'), (u'MIDDLE', u'EAST'), (u'Maya', u'Angelou'), (u'Sao', u'Paulo')]


Кластеризация с KMeans

In [33]:
# Признаки
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=500, min_df=10, ngram_range=(2, 2))
data_set = [text for text in texts_without_stopwords if len(text) > 10]
matrix = vectorizer.fit_transform(data_set)

In [55]:
# Кластеризуем
from sklearn.cluster import KMeans
k = 5
# Больше 5 было плохо, так как в разные кластеры попадались почти одинаковые сообщения, с одинаковой тематикой.
# На число кластеров довольно сильно влиял параметр ngram_range. Без учета биграмм, кластеры получались менее информативнее,
# было тяжелее определить темы.
model = KMeans(n_clusters=k)
preds = model.fit_predict(matrix.toarray())
print(preds[:30])

[3 3 2 3 3 3 2 3 2 3 3 3 3 3 3 3 3 2 3 2 3 2 3 2 3 3 3 2 3 3]


In [56]:
# Визуализация кластеров - сообщения, самые близкие к ядрам
num_of_samples = 10
for j in range(k):
    print "===================================="
    print str(j) + "CLASS:"
    d = model.transform(matrix.toarray())[:, j]
    ind = np.argsort(d)[::][:num_of_samples]
    for i in range(len(ind)):
        print "---------------------------"
        print data_set[ind[i]]

0CLASS:
---------------------------
currently office limited access email. need immediate assistance, please contact Nora Toiv,
toivnf@state.gov, Joanne Laszczych, laszczychj@state.gov, 202-647-5548.
If emergency, please call Operations Center reach me.
thank you.
cdm
---------------------------
currently traveling intermittent access email. need immediate assistance, please contact Nora Toiv
(toivnf@state.gov) 202-647-8633 Joanne Laszczych (Laszczych1@state.gov) 202-647-5548.
If emergency business hours, please reach Operations Center reach me.
Thank you
cdm
---------------------------
Government Travel Visas Government's Leaders
I currently office unavailable 6:00pm EST today, December 10. need immediate
assistance, please contact Nora Toiv, toivnf@state.gov, Joanne Laszczych, laszczychj@state.gov, 202-647-5548.
If emergency, please call Operations Center reach me.
thank you.
cdm
---------------------------
currently traveling outside country limited access email. need immediate assi

Оцениваю интерпретируемость с помощью одного ассесора.
Вопросы:
1. Опиши тему каждой группы
2. Согласен ли ты с данной кластеризацией на примере, показаном выше? Есть ли тексты в какой либо из групп, которые ты хочешь перенести в другие?
3. Нужно ли создать еще один класс?
4. Нужно ли одну класс убрать?


Ответы:
1. 1ая группа - автоответы. 2ая - Сообщения с расписаниями. 3ая- номера дел, документов, приказов, возможно запросы данных. 4ая- неформальная переписка. 5ая - перписка с Special Assistant Secretary
2. да, согласен
3. нет
4. нет

Также были даны сообщения(см.ниже), которые требовалось отнести к одному из кластеров. Измерялась скорость соотнесения. В среднем 45сек.
 

Качество можно подсчитываь 

B6
Thursday, March 3, 2011 9:45 PM
H: Latest Syria aiding Qaddafi more... Sid
hrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx
March 3, 2011
For: Hillary

Pis print.
-•-...-^
H < hrod17@clintonernailcom>
Wednesday, September 12, 2012 2:11 PM
°Russorv@state.gov'
Fw: Meet Right-Wing Extremist Behind Anti-fvluslim Film Sparked Deadly Riots
From [meat)
Sent: Wednesday, September 12, 2012 01:00 PM
To: 11
Subject: Meet Right Wing Extremist Behind Anti-Muslim Film Sparked Deadly Riots
htte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-
deadly-riots/
Sent Verizon Wireless 4G LTE DROID
U.S. Department State
Case No. F-2015-04841
Doc No. C05739559
Date: 05/13/2015
STATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM.
SUBJECT AGREEMENT SENSITIVE INFORMATION & REDACTIONS. FOIA WAIVER. STATE-5CB0045251


H <hrod17@clintonemail.corn>
Friday, March 11, 2011 1:36 PM
Huma Abedin
Fw: H: Latest: Syria aiding Qaddafi more... Sid
hrc memo Syria aiding libya 030311.docx
Pis print.

Anne-Marie Slaughter
Sunday, March 13, 2011 9:39 PM
Jacob Mills, Cheryl D; Reinesp@stategov; Abedin, Hurtle
piece libya nfz appear NYT op-ed page tomorrow
B6
wanted make sure heads up. AM
Anne-Marie Slaughter professor politics international affairs Princeton.
By Anne-Marie Slaughter
PRESIDENT Obama says noose tightening around Col. Muammar al-Qaddafi. fact, is
tightening around Libyan rebels, Colonel Qaddafi makes world's dithering and
steadily retakes rebel-held tow.ns, United States Europe temporizing no-flight zone
while Organization Islamic Conference, Gulf Cooperation Council Arab
League called United Nations Security Council authorize one. Opponents no-
flight zone put forth five main arguments, none which, closer examination, hold up.
It's interest. Gen. Wesley K. Clark argues "Libya doesn't sell much oil United
States" Americans "want support democratic movements region," are
already Iraq Afghanistan. Framing issue terms oil exactly Arab
populations indeed much world expect, cynical our
professions support democracy human rights. chance support new
beginning Muslim world — new beginning accountable governments provide
services opportunities citizens ways could dramatically decrease support for
terrorist groups violent extremism. It's hard imagine something strategic interest.
It counter-productive. Many thoughtful commentators, including Al Jazeera's director
general, Wadah Khanfar, argue important Arab spring coming
from Arabs themselves. perspective, Western military intervention play right into
Qaddati's hands, allowing broadcast pictures Western bombs falling Arab civilians. But
these arguments, important, must weighed appeals Libyan opposition
fighters asking international help, now, astonishingly, support no-flight zone by
some governments kept populations quiescent holding specter of
foreign intervention. Assuming no-flight zone imposed international coalition that
includes Arab states, opportunity establish new narrative Western support Arab
democrats.
U.S. Department State
Case No. F-2015-04841
Doc No. C05739568
Date: 05/13/2015
STATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM.
SUBJECT AGREEMENT SENSITIVE INFORMATION & REDACTIONS. FOIA WAIVER. STATE-SCB0045012
U.S. Department State
Case No. F-2015-04841
Doc No. C05739568
Date: 05/13/2015
STATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM.

 .....
From Randolph, Lawrence M
Sent: Wednesday, September 12, 2012 04:33 PM
To: Mills, Cheryl D
Subject: RE: dry eye NEA
Including mine. remarks really moving. Chriswas amazing man. huge loss. know, Libya
before coming almost ten years, never worked calm, cool headed, funny
diplomat. Made seem really easy- even one hardest places work world.

asked attend svtc today Embassy Tripoli, first met many staff went with
you Malta Tripoli reopening Embassy.
Today's deaths hit much harder would guessed. always proud serve you, never have
your words meaningful today's svtc. Every day tenure extraordinary, none
more today. Thank inspirational leadership example.
As ever,
Harold
U.S. Department State
Case No. F-2015-04841
Doc No. C05739571
Date: 05/13/2015
STATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM.
SUBJECT AGREEMENT SENSITIVE INFORMATION & REDACTIONS. FOIA WAIVER. STATE-SCB0045269

Hope. See picture Kamala Sent

Another photo.

Amazing.
Sullivan, Jacob J <Sullivanii@state,gov>
Wednesday, September 12, 2012 10:34 PM
Sherman, Wendy R; H; Mills, Cheryl D; Nuland, Victoria J; Reines, Philippe
Re: Youth Libya